# Facility Location with Regions

Based on Example 14.3 from the SAS Optimization documentation: https://go.documentation.sas.com/doc/en/pgmsascdc/default/casmopt/casmopt_milpsolver_examples03.htm

For a set of customers and sites, choose which sites to build such that:
- We minimize the sum of the distances between customers and their assigned sites and the building costs of sites
- The capacity for each site is not exceeded
- Sites and customers are in same region
- The budget for all sites built is not exceeded


#### Mixed Integer Linear Programming Formulation

$ \begin{array}{llllll} 
\min & \displaystyle \sum _{i \in L} \displaystyle \sum _{j \in F} c_{ij} x_{ij} &+& \displaystyle \sum _{j \in F} f_ j y_ j \\ 
\text{s.t.} & \displaystyle \sum _{j \in F} x_{ij} & = & 1 & \forall i \in L & \text{(assign\_def)} \\ 
& \displaystyle \sum _{i \in L} d_ i x_{ij} & \leq & Cy_ j & \forall j \in F & \text{(capacity)} \\ 
& x_{ij} & = & 0 & \forall i,j \text{ if } r_i \ne r_j & \text{(region\_con)} \\
\\
& \displaystyle \sum _{j \in F} f_ j y_ j &\le& B & \text{(budget\_con)}\\
\\
& x_{ij} \in \{ 0,1\} & & & \forall i \in L, j \in F \\
& y_{j} \in \{ 0,1\} & & & \forall j \in F 
\end{array} 
$


#### Input Data

For the input data we have a comma-separated value (CSV) file with all cities in Germany with more than 50,000 inhabitants retrieved from the German Federal Office of Statistics (www.destatis.de) that also includes geo locations of the cities for plotting. The following code reads the file and plots the data on a map.

In [1]:
import folium
import pandas as pd

# Read the input data and make sure the numbers are all parsed correctly, then print the top of the DataFrame
indata = pd.read_csv('cities_germany.csv', sep=';', decimal=',')
indata["size"] = pd.to_numeric(indata["size"].str.replace(" ", ""), errors='coerce')
indata["density"] = pd.to_numeric(indata["density"].str.replace(" ", ""), errors='coerce')
print(indata)

# Display the cities on a map of Germany
map_input = folium.Map(location=(52, 9), zoom_start=6)
for index, row in indata.iterrows():
    folium.Marker(location=[row["lat"], row["lon"]], tooltip=f'{row["name"]}<br>Size: {row["size"]}<br>Density: {row["density"]}').add_to(map_input)
display(map_input)

     state                            name     size  density  zipcode  \
0       11                   Berlin, Stadt  3755251     4214    10178   
1        2   Hamburg, Freie und Hansestadt  1892122     2506    20095   
2        9       München, Landeshauptstadt  1512491     4868    80331   
3        5                     Köln, Stadt  1084831     2678    50667   
4        6        Frankfurt am Main, Stadt   773068     3113    60311   
..     ...                             ...      ...      ...      ...   
190      1                 Elmshorn, Stadt    50772     2377    25335   
191      3                    Emden, Stadt    50535      450    26721   
192      3                   Goslar, Stadt    50203      306    38640   
193      5                  Willich, Stadt    50144      740    47877   
194      8  Heidenheim an der Brenz, Stadt    50025      467    89522   

           lon        lat  type  
0    13.405538  52.517670     1  
1     9.996970  53.550678     1  
2    11.575997  48.13

#### Connecting to SAS Viya

The connection options are provided in "cas.py" in the same directory. The following code establishes a connection and creates a SAS data set from the Pandas data frame.

In [2]:
import swat

sas = swat.CAS(hostname='https://my-cas-host.com:443/cas-shared-default-http/', pkce=True)
_ = sas.upload_frame(indata, casout={"name": "indata", "replace": True})


NOTE: Cloud Analytic Services made the uploaded file available as table INDATA in caslib CASUSER(phchri).
NOTE: The table INDATA has been created in caslib CASUSER(phchri) from binary data uploaded to Cloud Analytic Services.


#### Defining and Solving the Optimization Problem with OPTMODEL

In this specific example, every customer location (city) can also be a site. The demand of each city is its size while we use the density as the cost to build a site. This means that building a site in a less densely populated cities is preferable. This leads to an interesting optimization problem for demonstration purposes but has no real-world meaning.

First we define the OPTMODEL code that we want to execute. Then it is sent to SAS.


In [3]:
# Define the model in OPTMODEL
optmodel_code = """
   /* Define set of sites, only need one set since all sites are also customers */
   set <str> SITES;

   /* Latitude and Longitude for SITES */
   num lat {SITES};
   num lon {SITES};

   /* Capacity of each site */
   num C = 5000000;

   /* Budget for building cost */
   num B = 18000;

   /* Other parameters */
   num demand {SITES};
   num cost {SITES};
   num region {SITES};

   /* Define a set of tuples for all possible assignments */
   set PAIRS = {i in SITES, j in SITES: region[i] = region[j]};

   /* Compute distances between sites */
   num distance {<i,j> in PAIRS}
       = round(geodist(lat[i], lon[i], lat[j], lon[j], 'K'));

   /* Read the data */
   read data indata into SITES=[name] lat lon region=state demand=size cost=density;

   /* Create variables */
   var Assign {PAIRS} binary;
   var Build {SITES} binary;

   /* Define objective function */
   min TotalCost
       = sum {<i,j> in PAIRS} distance[i,j] * Assign[i,j]
         + sum {j in SITES} cost[j] * Build[j];

   /* Each site needs to be assigned to exactly once */
   con assign_def {i in SITES}:
      sum {<(i),j> in PAIRS} Assign[i,j] = 1;

   /* Each site we build can handle at most C demand */
   con capacity {j in SITES}:
      sum {<i,(j)> in PAIRS} demand[i] * Assign[i,j] <= C * Build[j];

   /* The cost for all sites build may not exceed the budget B */
   con budget:
      sum {j in SITES} cost[j] * Build[j] <= B;

   /* Solve with the MILP solver */
   solve;

   /* Create output data sets */
   create data assignments from
        [customer site]={<i,j> in PAIRS: Assign[i,j] > 0.5}
        lat1=lat[i] lon1=lon[i] lat2=lat[j] lon2=lon[j] distance[i,j];
   create data sites from
        [site]={j in SITES: Build[j] > 0.5}
            name=j lat[j] lon[j] cost[j];
"""
# Submit the model to SAS
sas.loadactionset("optimization")

_ = sas.runOptmodel(optmodel_code)

# Create output data frames to plot the solution
assignments = sas.CASTable("assignments")
sites = sas.CASTable("sites")

NOTE: Added action set 'optimization'.
NOTE: There were 195 rows read from table 'INDATA' in caslib 'CASUSER(phchri)'.
NOTE: Problem generation will use 16 threads.
NOTE: The problem has 7766 variables (0 free, 0 fixed).
NOTE: The problem has 7766 binary and 0 integer variables.
NOTE: The problem has 391 linear constraints (196 LE, 195 EQ, 0 GE, 0 range).
NOTE: The problem has 15532 linear constraint coefficients.
NOTE: The problem has 0 nonlinear constraints (0 LE, 0 EQ, 0 GE, 0 range).
NOTE: The OPTMODEL presolver is disabled for linear problems.
NOTE: The initial MILP heuristics are applied.
NOTE: The MILP presolver value AUTOMATIC is applied.
NOTE: The MILP presolver removed 8 variables and 8 constraints.
NOTE: The MILP presolver removed 16 constraint coefficients.
NOTE: The MILP presolver modified 116 constraint coefficients.
NOTE: The presolved problem has 7758 variables, 383 constraints, and 15516 constraint coefficients.
NOTE: The MILP solver is called.
NOTE: The parallel Branc

Print the parts of the objective. Note that the budget is not exceeded.

In [4]:
# Print the sum of the distances
total_distance = assignments["distance"].sum()
print(f"Total distance: {total_distance}")

# Print the sum of the building costs
total_site_cost = sites["cost"].sum()
print(f"Total site cost: {total_site_cost}")

print(f"Objective: {total_distance + total_site_cost}")

Total distance: 10225.0
Total site cost: 17930.0
Objective: 28155.0


Display the solution on a map.

In [5]:
map_region = folium.Map(location=(52, 9), zoom_start=6)

# Plot all cities
for index, row in indata.iterrows():
    folium.Marker(location=[row["lat"], row["lon"]], tooltip=f'{row["name"]}<br>Size: {row["size"]}<br>Density: {row["density"]}').add_to(map_region)

# Plot the cities that are sites to build
for index, row in sites.iterrows():
    folium.Marker(location=[row["lat"], row["lon"]], tooltip=row["name"], icon=folium.Icon(color="green")).add_to(map_region)

# Plot the assignments of customers to sites
for idx, row in assignments.iterrows():
    folium.PolyLine([[row["lat1"], row["lon1"]],
                     [row["lat2"], row["lon2"]]]).add_to(map_region)

map_region

# Using Dantzig-Wolfe Decomposition

The following code defines a block structure and calls the Dantzig-Wolfe decomposition algorithm (DECOMP) to solve this problem. The constraint `budget` links together the otherwise independent blocks, hence it's a good idea to try DECOMP with user defined blocks. While DECOMP takes longer to solve this particular problem to optimality, it finds a good feasible (often optimal) solution very quickly. For larger, real world problems this is typically more noticeable than for a small example like this.

In [6]:
optmodel_code = optmodel_code.replace("solve;","""
   /* Assign blocks for DECOMP */
   for {j in SITES} do;
      assign_def[j].block = region[j];
      capacity[j].block = region[j];
   end;

   solve with milp / maxtime=30 decomp;
""")
_ = sas.runOptmodel(optmodel_code)


NOTE: There were 195 rows read from table 'INDATA' in caslib 'CASUSER(phchri)'.
NOTE: Problem generation will use 16 threads.
NOTE: The problem has 7766 variables (0 free, 0 fixed).
NOTE: The problem has 7766 binary and 0 integer variables.
NOTE: The problem has 391 linear constraints (196 LE, 195 EQ, 0 GE, 0 range).
NOTE: The problem has 15532 linear constraint coefficients.
NOTE: The problem has 0 nonlinear constraints (0 LE, 0 EQ, 0 GE, 0 range).
NOTE: The remaining solution time after problem generation and solver initialization is 29,90 seconds.
NOTE: The initial MILP heuristics are applied.
NOTE: The MILP presolver value AUTOMATIC is applied.
NOTE: The MILP presolver removed 8 variables and 8 constraints.
NOTE: The MILP presolver removed 16 constraint coefficients.
NOTE: The MILP presolver modified 116 constraint coefficients.
NOTE: The presolved problem has 7758 variables, 383 constraints, and 15516 constraint coefficients.
NOTE: The MILP solver is called.
NOTE: The Decompositio

In [7]:
# Close the connection to swat
sas.close()